In [ ]:
import sys
import os
import numpy as np
from pathlib import Path
from datetime import datetime
import importlib  # For reloading modules during testing

# Add parent directory to path for imports
notebook_dir = Path.cwd()
project_root = notebook_dir.parent
sys.path.append(str(project_root))

# First import the modules
import aicons.definitions.aicon
import aicons.bayesbrainGPT.utility_function.utility_base  # Import the package instead
import aicons.bayesbrainGPT.decision_making.action_space

# Now reload them for development/testing
importlib.reload(aicons.definitions.aicon)
importlib.reload(aicons.bayesbrainGPT.utility_function.utility_base)  # Reload the package
importlib.reload(aicons.bayesbrainGPT.decision_making.action_space)

# Import the classes after reloading
from aicons.definitions.aicon import AIcon

In [59]:
# Create the AIcon
aicon = AIcon("marketing_aicon")
print(f"AIcon created: {aicon.name}")

AIcon created: marketing_aicon


### Defining Action Space

In [60]:
# Define an action space
aicon.define_action_space(
    space_type='marketing',
    total_budget=1000.0,
    num_ads=2,
    budget_step=100.0,
    ad_names=['google', 'facebook']
)

ActionSpace(dimensions=2, constraints=1)

In [61]:
print(aicon.brain.action_space.raw_print()) 

dimensions: ['google_budget: [0.0, 1000.0] step=100.0', 'facebook_budget: [0.0, 1000.0] step=100.0']
constraints: ['<function create_marketing_ads_space.<locals>.budget_sum_constraint at 0x1691e8ea0>']
actions: [{'google_budget': 700.0, 'facebook_budget': 300.0}, {'google_budget': 800.0, 'facebook_budget': 200.0}, {'google_budget': 100.0, 'facebook_budget': 900.0}, {'google_budget': 600.0, 'facebook_budget': 400.0}, {'google_budget': 400.0, 'facebook_budget': 600.0}, {'google_budget': 600.0, 'facebook_budget': 400.0}, {'google_budget': 700.0, 'facebook_budget': 300.0}, {'google_budget': 600.0, 'facebook_budget': 400.0}, {'google_budget': 600.0, 'facebook_budget': 400.0}, {'google_budget': 300.0, 'facebook_budget': 700.0}, {'google_budget': 400.0, 'facebook_budget': 600.0}, {'google_budget': 500.0, 'facebook_budget': 500.0}, {'google_budget': 800.0, 'facebook_budget': 200.0}, {'google_budget': 600.0, 'facebook_budget': 400.0}, {'google_budget': 700.0, 'facebook_budget': 300.0}, {'google

### Defining Utility Function

In [62]:
utility = aicon.define_utility_function(
    utility_type='weighted_sum',
    weights={'google': 0.6, 'facebook': 0.4}
)

Error creating utility function: WeightedSumUtility.__init__() got an unexpected keyword argument 'action_space'


### Defining State Rep State

In [ ]:
# Add state factors
aicon.add_state_factor(
    name='base_conversion_rate',
    factor_type='continuous',
    value=0.05,
    uncertainty=0.01
)

aicon.add_state_factor(
    name='primary_channel',
    factor_type='categorical',
    value='google',
    categories=['google', 'facebook'],
    probs=[0.7, 0.3]
)